In [1]:
from torchtext.datasets import IMDB
from torchtext.vocab import FastText
from torchtext import data, datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from IPython.core.debugger import set_trace

In [2]:
TEXT = data.Field(lower=True,include_lengths=False,batch_first=False)
LABEL = data.Field(sequential=False,)
train, test = datasets.IMDB.splits(TEXT,LABEL)
TEXT.build_vocab(train, vectors=FastText(language = 'en'))
LABEL.build_vocab(train)

In [3]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=16, device='cuda',shuffle=True)

In [ ]:
batch = next(iter(train_iter))

In [ ]:
weight_matrix = TEXT.vocab.vectors
n_tokens = weight_matrix.size(0)

In [ ]:
class ClassifierModel(nn.Module):
    
    def __init__(self, ntoken, ninp,
                 nhid, nlayers, bsz, noutputs,
                 dropout=0.5):
        super(ClassifierModel, self).__init__()
        self.nhid, self.nlayers, self.bsz = nhid, nlayers, bsz
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.output = nn.Linear(nhid,noutputs)

        self.init_weights()
        self.hidden = self.init_hidden(bsz)
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.output.bias.data.fill_(0)
        self.output.weight.data.uniform_(-initrange, initrange)
 
    def forward(self, input):
        emb = self.drop(self.encoder(input))
        self.init_hidden(self.bsz)
        output, self.hidden = self.rnn(emb, self.hidden)
        output = self.drop(output)[-1]
        output = self.output(output)      
        return output
 
    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return (torch.tensor(weight.new(self.nlayers, bsz, self.nhid).zero_().cuda()),
                torch.tensor(weight.new(self.nlayers, bsz, self.nhid).zero_()).cuda())


weight_matrix = TEXT.vocab.vectors
model = ClassifierModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 3, 16,2)
model = model.cuda()
model.encoder.weight.data.copy_(weight_matrix)

In [ ]:
model_state = model.state_dict()
pretrained_state = torch.load('lstm_8.pt')
pretrained_state = { k:v for k,v in pretrained_state.items() if k in model_state and v.size() == model_state[k].size() }
model_state.update(pretrained_state)
model.load_state_dict(model_state)

In [ ]:
for index, batch in enumerate(train_iter):
    pred = model(batch.text)
    print(pred)